## Missing Data - TimeSeries

There are a few strategies we can choose from and they each have their pros/cons:
1) **Fill with a Relevant Value**  
 - If we know what the nulls 'should' be, easy to fill them with that value",  
 - For numbers, perhaps the null indicates a 0",  
 - Or, for string columns, might be easier to handle if we fill with \"Missing\" or \"Unknown\"
 
2) **Fill with a Reasonable Value**   
 - For numeric data, it might be acceptable to fill with a measure of central tendency (mean or median),
 - For categorical/string data, might be acceptable to fill with the most common (mode),
 - But beware! Filling in missing values can lead to you drawing incorrect conclusions. If most of the data from a column are missing, it's going to appear that the value you filled it in with is more common that it actually was!  

3) **Specify Missing Data**  
 - If you plan to fill in missing values, it might make sense to specify that the data was originally missing by creating a new indicator column,
 - This can be helpful when you suspect that the fact the data was missing could be important for an analysis.

4) **Drop Missing Data**  
 - While you should try to keep as much relevant data as possible, sometimes the other methods don't make as much sense and it's better to remove or **drop** the missing data,
 - We typically drop missing data if very little data would be lost and/or trying to fill in the values wouldn't make sense for our use case  

In order of preference, we go with 1, 3, 2, and 4.

### Any time we fill a null value with a mathematical process, we are creating data. This means that any resulting insights need to be reported with this in mind.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

pd.set_option('display.max_rows', 5000)
plt.rcParams["figure.figsize"] = [10.5, 10.5]
plt.rcParams["figure.autolayout"] = True

In [ ]:
apc = pd.read_parquet('./data/Flat_Iron_Dataset_06292023.parquet')

In [ ]:
apc.head()

In [ ]:
apc['sample_date'] = pd.to_datetime(apc['sample_date'])

In [ ]:
wrkg = apc[apc["fldptag"]=="2002000001841276"]

In [ ]:
wrkg

In [ ]:
wrkg = wrkg.set_index('sample_date')

In [ ]:
wrkg = wrkg.pivot(columns='endpoint_id', values='voltage')

In [ ]:
wrkg

In [ ]:
wrkg.isnull().sum()

Looks like only one of our customers has a full dataset. Let's take a look.

In [ ]:
sns.heatmap(wrkg.isnull())

In [ ]:
raw_series = wrkg.iloc[:, 16]

In [ ]:
raw_series

In [ ]:
raw_series.plot()

The challenge with `interpolate` is that it can't go from nothing. It needs to have a starting point and an ending point. In other words, it's only effective for gaps in data, not whole cloth creation.

The data we've got here, as we know, has enormous gaps in it that begin at the first timestep for almost all of the observations. So we need to create some gaps in our data, and we're going to do that with resampling. Fortunately, there's built-in resampling in pandas.

Two types of resampling are:

**Upsampling**: Where you increase the frequency of the samples, such as from minutes to seconds.  
**Downsampling**: Where you decrease the frequency of the samples, such as from days to months.

Our data are in fifteen minute increments. We need to downsample it, and then upsample it in order to create some null values. Since we're downsampling, we also need to provide a way to consolidate the data, using mean in this example


In [ ]:
downsample = raw_series.resample("D").mean()

|Rule | Meaning |
|--|:--|
|B       |  business day frequency|
|C      |   custom business day frequency (experimental)|
|D     |    calendar day frequency|
|W    |     weekly frequency|
|M   |      month end frequency|
|SM |       semi-month end frequency (15th and end of month)|
|BM  |      business month end frequency|
|CBM     |  custom business month end frequency|
|MS     |   month start frequency|
|SMS   |    semi-month start frequency (1st and 15th)|
|BMS  |     business month start frequency|
|CBMS|      custom business month start frequency|
|Q |        quarter end frequency|
|BQ|        business quarter endfrequency|
|QS|        quarter start frequency|
|BQS|       business quarter start frequency|
|A|         year end frequency|
|BA, BY  |  business year end frequency|
|AS, YS  |  year start frequency|
|BAS, BYS | business year start frequency|
|BH       | business hour frequency|
|H        | hourly frequency|
|T, min  |  minutely frequency|
|S      |   secondly frequency|
|L, ms |    milliseconds|
|U, us|     microseconds|
|N  |       nanoseconds|

In [ ]:
downsample

In [ ]:
downsample.plot()

In [ ]:
upsample = downsample.resample("H").asfreq()

In [ ]:
upsample

We've now got a series witha bunch of nulls.

In [ ]:
series = upsample.copy()

The `interpolate` method in pandas has a truly ridiculous number of options for the `method` parameter:

> ‘linear’: Ignore the index and treat the values as equally spaced. This is the only method supported on MultiIndexes.

> ‘time’: Works on daily and higher resolution data to interpolate given length of interval.

> ‘index’, ‘values’: use the actual numerical values of the index.

> ‘pad’: Fill in NaNs using existing values.

>‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘barycentric’, ‘polynomial’: Passed to scipy.interpolate.interp1d, whereas ‘spline’ is passed to scipy.interpolate.UnivariateSpline. These methods use the numerical values of the index. Both ‘polynomial’ and ‘spline’ require that you also specify an order (int), e.g. df.interpolate(method='polynomial', order=5). Note that, slinear method in Pandas refers to the Scipy first order spline instead of Pandas first order spline.

> ‘krogh’, ‘piecewise_polynomial’, ‘spline’, ‘pchip’, ‘akima’, ‘cubicspline’: Wrappers around the SciPy interpolation methods of similar names. See Notes.

> ‘from_derivatives’: Refers to scipy.interpolate.BPoly.from_derivatives which replaces ‘piecewise_polynomial’ interpolation method in scipy 0.18.

In [ ]:
lin_series = series.interpolate("linear")

`Linear` treats the spaces as if they are equal. In this case, they are equal, so there's no difference than with `time` or `index`

In [ ]:
lin_series

In [ ]:
lin_series.plot()

In [ ]:
time_series = series.interpolate("time")

In [ ]:
time_series

In [ ]:
time_series.plot()

`Polynomial` fits a polynomial of order _k_

In [ ]:
poly_series = series.interpolate('polynomial', order=3)

In [ ]:
poly_series.plot()

`Spline` uses spline interpolation, fitting a piecewise polynomial to the intervening space.

In [ ]:
spline_series = series.interpolate('spline', order=3)

In [ ]:
spline_series.plot()

In [ ]:
deriv_series = series.interpolate('from_derivatives')

In [ ]:
deriv_series.plot()

Realistically, interpolate can do the job pretty well, so long as the job is filling in gaps in the data. It's no good when it comes to creating data from whole cloth. If we want to really fill in from the start, I would suggest the following process:

1) Calculate the diffs for the data that you have. `pd.diff` will do the job.  
2) Use the diffs to generate a distribution.  
3) Use this distribution to generate a vector of random values for the appropriate timestamps, using the mean of the known timeseries as a starting point.  

Obviously, this assumes that the missing time series follows the same distribution as the known time series.